In [ ]:
#!/usr/bin/python
# Modified 11/29/2019 by HYK
# dilution model:  plot after 
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!
# First let system reach steady state
# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen
"""
import roadrunner
import numpy 
from numpy import random as rand
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function
def doSimulation(tc,nc):      # Here we include signal as we turn it on only after a few cell cycles
    r = loada("""
    # define cell cycling parameters
    TCycle = 10                                    
    V := (time/TCycle + 1)
    
    #define ode
    # should we include positive feedback additively
    R1: pX => ; k*S*pX        # repression of promoter X 
    R2: => X ; V*alphaX*pX    # X is synthesized but repressed b 
    R3: X => ; X*deltaX                  # degradation of X
    R4: => Y ; V*alphaY*(Ky)^N/((X/V)^N+Ky^N)  # Y synthesis repressed by X
    R5: Y => ; Y*deltaY
   
   #define initial values
    pX = 1            # initial state of the promoter
    S = 0             # the concentration of signal * (normalized units) 
    X = 700                 # the initial number of copies of X
    Y = 0                 # the initial number of copies of Y
 
    alphaX = 10^3          # synthesis rate of X (copies/hr)
    alphaY = 10^3          # synthesis rate of Y (copies/hr)
    deltaX = 1             # degradation rate (/hr)
    deltaY = 1             # degradation rate (1/hr)
    k = 0.02               # silencing rate per unit signal
    Ky = 40              # MM constant for activation of Y by X
    N = 10                 # Hill coefficient for activation of Y by X
    """)
    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time 
    
    ############################
    dt = 0.1;     # time interval for plotting
    result=[]
    for i in range(0,nc,1):       # for loop over the total number of cell cycles
        if i == 0:
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])
            for row in SubResult:
                result.append(row)
        else:
            r.X = rand.binomial(r.X, 0.5)   # randomly partition
            r.Y = rand.binomial(r.Y, 0.5)
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])             
            for row in SubResult:
                row[0]=row[0]+(i*tc)     # update the time vector
                result.append(row)
            if (i==4):       # if 5 cell cycles have elapsed
                r.S = 1      # initiate the repressive signal
    result = ar(result)
    return result

#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)


In [ ]:
for i in range(0,20,1):
    print(i)
    result = doSimulation(10,25)    
    writeCSV('data/data_{}.csv'.format(i),result)

